In [2]:
# download dataset
!wget https://s3.us-east-2.amazonaws.com/mrqa/release/v2/train/NewsQA.jsonl.gz
!wget https://s3.us-east-2.amazonaws.com/mrqa/release/v2/dev/NewsQA.jsonl.gz

--2023-01-19 06:07:03--  https://s3.us-east-2.amazonaws.com/mrqa/release/v2/dev/NewsQA.jsonl.gz
Resolving s3.us-east-2.amazonaws.com (s3.us-east-2.amazonaws.com)... 52.219.97.137
Connecting to s3.us-east-2.amazonaws.com (s3.us-east-2.amazonaws.com)|52.219.97.137|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3142984 (3.0M) [binary/octet-stream]
Saving to: ‘NewsQA.jsonl.gz’

NewsQA.jsonl.gz     100%[===================>]   3.00M  4.47MB/s    in 0.7s    

2023-01-19 06:07:04 (4.47 MB/s) - ‘NewsQA.jsonl.gz’ saved [3142984/3142984]



In [1]:
!pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 87.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 86.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 18.6 MB/s eta 0:00:00


In [2]:
# load dataset
import pandas as pd    
train_dataset = pd.read_json(path_or_buf='/content/drive/MyDrive/newsQ&Adataset/training/NewsQA.jsonl.gz', lines=True)
test_dataset = pd.read_json(path_or_buf='/content/drive/MyDrive/newsQ&Adataset/test/NewsQA.jsonl.gz', lines=True)


In [3]:
#clean up
train_dataset = train_dataset[1:].drop(['header'],axis=1)
test_dataset = test_dataset[1:].drop(['header'], axis=1)

In [4]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 18.7 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [5]:
# pre process
def get_cqa(dataset):
  contexts=[]
  questions=[]
  answers =[]
  for index, example in dataset.iterrows():
            context = example['context']
            for qa in exAutoModelForQuestionAnsweringample['qas']:
                question = qa['question']
                for answer,detected_answer in zip(qa['answers'],qa['detected_answers']):
                    contexts.append(context)
                    questions.append(question)
                    answers.append({'text':answer,'char_span':detected_answer['char_spans'][0]})
  return {"context":contexts,"question":questions,"answers":answers}
  # return [contexts,questions,answers]


In [6]:
from datasets import Dataset
test_df = pd.DataFrame(get_cqa(test_dataset))
test_ds = Dataset.from_pandas(test_df, split="test")

In [7]:
#load tokenizer
from transformers import AutoTokenizer

model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [8]:
#preporcess training daata into a model compatible input and label
max_length = 384
stride = 128


def preprocess_training_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["char_span"][0]
        end_char = answer["char_span"][1]
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [27]:
def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [9]:
train_dataset_f = test_ds.map(
    preprocess_training_examples,
    batched=True,
    remove_columns=test_ds.column_names,
)

  0%|          | 0/5 [00:00<?, ?ba/s]

In [10]:
#loading models


from transformers import AutoModelForQuestionAnswering
model = AutoModelForQuestionAnswering.from_pretrained('/content/drive/MyDrive/bert-finetuned-newsQA/checkpoint-1404')




In [13]:
# set training arguements
from transformers import TrainingArguments

args = TrainingArguments(
    "bert-finetuned-newsQA",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=True,
)




In [42]:
train_dataset_f

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 11231
})

In [15]:
#train model
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset_f,
    # eval_dataset=validation_dataset,
    tokenizer=tokenizer,
)
trainer.train()

Using cuda_amp half precision backend


In [75]:
inputs = tokenizer(
        ["who is  the organizer of the head of Tennis Australia"],
        ["Last week, Craig Tiley, the head of Tennis Australia – the organizers of the Australian Open – told Australia’s Herald Sun that people who unreasonably taunt Djokovic would be removed from venues: “If they disrupt the enjoyment of anyone else – boom, they are out."],
        max_length=512,
        truncation="only_second",
        stride=stride,
        return_offsets_mapping=True,
        padding="max_length",
    )

In [76]:
offset_mapping=inputs.pop('offset_mapping')


In [ ]:
offset_mapping

In [ ]:
inputs

In [77]:
from datasets import Dataset
dataset = Dataset.from_dict(inputs)

In [78]:
dataset

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1
})

In [79]:
predictions, _, _ = trainer.predict(dataset)

***** Running Prediction *****
  Num examples = 1
  Batch size = 8


In [80]:
start_logits,end_logits=predictions

In [81]:
import numpy as np

In [82]:
start_indexes = np.argsort(start_logits[0])[-1 : -20 - 1 : -1].tolist()
end_indexes = np.argsort(end_logits[0])[-1 : -20 - 1 : -1].tolist()

In [83]:
start_indexes

[15, 0, 30, 16, 29, 23, 31, 18, 25, 17, 24, 14, 28, 48, 27, 34, 1, 40, 20, 21]

In [84]:
end_indexes

[18, 0, 17, 31, 15, 16, 24, 32, 19, 50, 23, 30, 35, 10, 1, 25, 29, 317, 38, 27]

In [ ]:
answers=[]

for start_index in start_indexes:
            for end_index in end_indexes:
                # Skip answers that are not fully in the context
                if offsets[0][start_index] is None or offsets[0][end_index] is None:
                    continue
                # Skip answers with a length that is either < 0 or > max_answer_length.
                if (
                    end_index < start_index
                    or end_index - start_index + 1 > 50
                ):
                    continue
                
                answers.append(
                    {
                        "text": raw['context'][0][offsets[0][start_index][0] : offsets[0][end_index][1]],
                        "logit_score": start_logits[0][start_index] + end_logits[0][end_index],
                    }
                )

In [85]:
context = "Last week, Craig Tiley, the head of Tennis Australia – the organizers of the Australian Open – told Australia’s Herald Sun that people who unreasonably taunt Djokovic would be removed from venues: “If they disrupt the enjoyment of anyone else – boom, they are out."

In [88]:
answer = context[offset_mapping[0][start_indexes[0]][0] : offset_mapping[0][end_indexes[0]][1]]

In [89]:
answer

'Craig Tiley'